# Interfacing Python and C/C++
- [doc](https://docs.python.org/3.5/extending/)

# calling C/C++ from Python
- also known as FFI, Foreign Function Interface
- works fine, but mistakes in C can corrupt the Python environment, causing
mysteries and crashes
- will show examples of calling 'libc' functions, which 'everything' uses
- to call your own C code, build a shared library and load it

In [ ]:
import ctypes

In [2]:
# Load the standard C library - full of routines all programs use
# On linux, this call would be 
# libc = cdll.LoadLibrary("libc.so")
# call below works on a mac
# windows?

lc = ctypes.cdll.LoadLibrary("libc.dylib")
lc

<CDLL 'libc.dylib', handle 1075d8608 at 0x107df1390>

In [3]:
# now have access to everything in the library,
# but takes some effort to call things correctly

[lc.strcmp, lc.printf, lc.malloc, lc.sin, lc.time]

In [4]:
# None means no args
# seconds since 1970

lc.time(None)

1571269540

# Call sin in libc
- sin takes and returns doubles
```
NAME
     sin -- sine function

SYNOPSIS
     #include <math.h>

     double
     sin(double x);

     long double
     sinl(long double x);

     float
     sinf(float x);
```

In [5]:
# ultimately calls libc sin routine

import math
math.sin(.5)

0.479425538604203

In [6]:
# get libc.sin function pointer

s = lc.sin
s

In [7]:
# this won't work

s(.5)

ArgumentError: argument 1: <class 'TypeError'>: Don't know how to convert parameter 1

In [8]:
# have to convert Python 'float' into C 'double',
# but it still won't work...get garbage result

s(ctypes.c_double(.5))

1022

In [9]:
# ...have to specify how to convert C return double 
# type back into Python float, by adding an
# attribute to the function object

s.restype = ctypes.c_double
s(ctypes.c_double(.5))

0.479425538604203

In [10]:
# looks like same routine is being called

s(ctypes.c_double(.5)) - math.sin(.5)

0.0

# C can call back into Python
- run C's qsort, with a Python sort predicate

In [ ]:
# this makes an integer C array class

IntArray5 = ctypes.c_int * 5

# make array object
ia = IntArray5(5, 1, 7, 33, 99)
qsort = lc.qsort
qsort.restype = None

# write the comparsion function in Python

def qsortCmp(a, b):
    print("qsortCmp", a[0], b[0] )
    return a[0] - b[0]

# declaration for comparison function
CMPFUNC = ctypes.CFUNCTYPE(ctypes.c_int, ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_int))

qsort(ia, len(ia), ctypes.sizeof(ctypes.c_int), CMPFUNC(qsortCmp)) 

In [ ]:
# list has been sorted by libc.qsort

list(ia)

# struct - lays out fields like C 'struct' would
- hardware interfaces often need precise byte layouts, aligned on certain boundaries
- does padding like a C struct would
- might use this to talk to an arduino or a robot
- [doc](https://docs.python.org/3.5/library/struct.html)

In [ ]:
from struct import *

In [ ]:
# 2 ints and a byte - why is len(p) 
# 12 bytes instead of 9?
# f is a format spec - 
# what types of things are going in the struct?

f = 'ici'
p = pack(f, 2,b'X', 3)
[p, len(p), unpack(f, p)]

In [ ]:
f = 'ihi'
p = pack(f, 4, 5, 6)
[p, len(p), unpack(f, p)]

In [ ]:
list(map(type, unpack(f, p)))

# Embedding Python In a C/C++ application
- can be incredibly useful
- not too hard, but not trivial 
- mostly consists of converting C and Python data types back and forth
- [doc](https://docs.python.org/3.5/extending/index.html)

# Example - Blender
- Blender is an open source animation system
- Pasting and running running the code below modifies the position of one vertex in the default cube
- Pretty much every operation in the GUI is available in the Python API
    - you can see the function in the tool tips
- allows programs to build 3D objects and automate animations 
- zoom with cntl-two-fingers

```
import bpy
bpy.data.objects["Cube"].data.vertices[0].co.x += 1.0
```